In [ ]:
# Collection of ideas from Graham Dumpleton writing
"""When implementing decorators you should be on the lookout for:
- Preserving introspection for the wrapped function
- Do not mess with how the Python Object model works

Ideal wrappers are therefore transparent.

Decorator syntax in Python is just a short hand way of being able 
to apply a wrapper around an existing function, while the definition
of function is being set up.

Monkey Patching achieves pretty much the same outcome, but is 
applied retrospectively from a different context after the original
function was created."""

In [ ]:
# General wrapper implemented with Class looks like this

class function_wrapper:
    def __init__(self, wrapped):
        self.wrapped = wrapped
        
    def __call__(self, *args, **kwargs):
        return self.wrapped(*args, **kwargs)

@function_wrapper
def function():
    pass

"""When decorated function gets invoked it is actually the __call__()
method of the wrapper object which is invoked. This in turn calls, 
wrapped function.

If you use normal function based decorator the nested function 
doesn't actually get passed explicitly."""


In [2]:
# Introspecting a wrapper

def function_wrapper(wrapped):
    def _wrapper(*args, **kwargs):
        return wrapped(*args, **kwargs)
    return _wrapper

@function_wrapper
def function():
    """A demo function"""
    pass

print(function.__name__)

_wrapper


In [ ]:
# This no longer works, implementing with class produces AttributeError
# as it has no attribute __name__

In [4]:
# Of course we can copy it manually

def function_wrapper(wrapped):
    def _wrapper(*args, **kwargs):
        return wrapped(*args, **kwargs)
    _wrapper.__name__ = wrapped.__name__
    _wrapper.__doc__ = wrapped.__doc__   
    return _wrapper

@function_wrapper
def function():
    """A demo function"""
    pass

print(function.__name__, function.__doc__)

function A demo function


In [9]:
"""
This of course get laborious and more attributes need to be copied
so stdlib provided functool.wraps decorator for that purpose to be
used with functions and functool.update_wrapper function
"""

import functools

class function_wrapper:
    
    def __init__(self, wrapped):
        self.wrapped = wrapped
        functools.update_wrapper(self, wrapped)
        
    def __call__(self, *args, **kwargs):
        return self.wrapped(*args, **kwargs)

# However these methods do not preserve the full arg specificatioin

import inspect

@function_wrapper
def function(arg1, arg2='def'):
    pass

In [10]:
inspect.getfullargspec(function)

FullArgSpec(args=['self'], varargs='args', varkw='kwargs', defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={})

In [11]:
inspect.signature(function)

<Signature (arg1, arg2='def')>

In [12]:
inspect.getsource(function)

"@function_wrapper\ndef function(arg1, arg2='def'):\n    pass\n"

In [ ]:
# Functools.update_wrapper preserve only naive introspection